In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chesss/Chess.pdf


In [2]:
!pip install wikipedia


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=48bf76e616fa751047b676e5ee0caea4d4b50a4be3a59b6d8dfd4928922a6b15
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [3]:
import google.generativeai as genai  # <-- this is key: use `google.generativeai`, not just `google`
from kaggle_secrets import UserSecretsClient
from IPython.display import Markdown
from google.api_core import retry

# Retry logic
is_retriable = lambda e: (isinstance(e, genai.types.generation_types.GenerationError) and e.code in {429, 503})

# Configure API Key
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# Load Gemini model
model = genai.GenerativeModel("gemini-2.0-flash")


In [18]:
!pip install PyMuPDF --quiet  # For PDF reading

import fitz  # PyMuPDF
import re

def extract_pdf_text(filepath):
    doc = fitz.open(filepath)
    text = ""
    for page in doc:
        text += page.get_text()
    return text


def split_sections(full_text):
    # Split into section title + content pairs
    sections = re.split(r'\n(Section \d+[.:].*?)\n', full_text)

    section_dict = {}
    seen_titles = set()

    for i in range(1, len(sections), 2):
        raw_title = sections[i].strip()
        content = sections[i + 1].strip()

        # Skip TOC-style entries (with many dots or page numbers at end)
        if re.search(r'\.{5,}', raw_title) or re.search(r'\d{1,3}$', raw_title):
            continue

        # Normalize to avoid duplicate titles
        normalized_title = re.sub(r'\s+', ' ', raw_title).strip()
        if normalized_title not in seen_titles:
            section_dict[raw_title] = content
            seen_titles.add(normalized_title)

    return section_dict


pdf_text = extract_pdf_text("/kaggle/input/chesss/Chess.pdf")  # Or use full path
sections = split_sections(pdf_text)

print("Available Sections:")
for key in sections:
    print("-", key)

def summarize_section(title):
    content = sections.get(title)
    if not content:
        return "❌ Section not found."

    prompt = f"""You are a chess coach assistant.
Here is a section of a chess curriculum titled: "{title}".

\"\"\"
{content[:4000]}
\"\"\"

Please provide a clear, beginner-friendly summary of this section.
"""
    response = model.generate_content(prompt)
    return response.text


def ask_about_section(title, question):
    content = sections.get(title)
    if not content:
        return "❌ Section not found."

    prompt = f"""You are a chess tutor AI.
Below is part of a chess training curriculum titled: "{title}".

\"\"\"
{content[:4000]}
\"\"\"

Answer the following question, using ONLY the information above:
Q: {question}
"""
    response = model.generate_content(prompt)
    return response.text


Available Sections:
- Section 1. The Rules of the Game and a Few Checkmates
- Section 2. The Opening
- Section 3: The Middlegame and Tactics
- Section 4: The Endgame, and More Checkmates
- Section 5: Playing in Tournaments


In [19]:
section_title = "Section 1. The Rules of the Game and a Few Checkmates"

# Summarize
summary = summarize_section(section_title)
print("🔎 Summary:\n", summary)

# Ask a Question
question = "What is checkmate"
answer = ask_about_section(section_title, question)
print("\n💬 Answer:\n", answer)


🔎 Summary:
 Okay, here's a simplified summary of the chess curriculum section, designed for beginners:

**Welcome to Chess!**

*   Chess is a super popular strategy game where two armies battle on a board.
*   The goal is to trap your opponent's King so it can't escape. This is called **checkmate**.
*   Don't worry if it seems complicated at first – we'll start with the basics!
*   Chess is fun for everyone, no matter how smart you are!
*   Chess will help you make better decisions!

**The Chessboard**

*   The board has 64 squares, with light and dark squares.
*   **Ranks** are the rows, numbered 1 to 8.
*   **Files** are the columns, labeled a to h.
*   Each square has a name like "e4" (file then rank).
*   White pieces start on ranks 1 and 2, Black on ranks 7 and 8.
*   Make sure a white square is in the bottom right corner of the board.
*   The center squares (d4, e4, d5, e5) are the most important - "downtown"!

**The Chess Pieces**

*   Let's learn the pieces: Rook, Bishop, Queen

In [20]:
# 📦 Install requirements
!pip install PyMuPDF wikipedia --quiet

# 📚 Imports
import fitz  # For PDF reading
import re
import wikipedia
from IPython.display import display, Markdown
from google import generativeai as genai
from kaggle_secrets import UserSecretsClient

# 📖 Load and parse PDF
def extract_pdf_text(filepath):
    doc = fitz.open(filepath)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def split_sections(full_text):
    sections = re.split(r'(Section \d+.*?)\n', full_text)
    section_dict = {}
    for i in range(1, len(sections), 2):
        title = sections[i].strip()
        content = sections[i+1].strip()
        section_dict[title] = content
    return section_dict



pdf_text = extract_pdf_text("/kaggle/input/chesss/Chess.pdf")
sections = split_sections(pdf_text)

few_shot_examples = """
Example 1:
Section Text:
The King is not allowed to move into a position where it would be under attack. The king must escape check when in danger.

Question:
How does the King move?

Answer:
The King moves one square in any direction but cannot move into check.

Example 2:
Section Text:
A pawn moves forward but captures diagonally. On its first move, it may advance two squares.

Question:
How does a pawn capture pieces?

Answer:
Pawns capture diagonally one square forward.

Example 3:
Section Text:
The Queen can control any number of squares vertically, horizontally, or diagonally.

Question:
How does the Queen move?

Answer:
The Queen can move in any direction—vertically, horizontally, or diagonally—for any number of squares.
"""

# 🧠 Few-shot-based answer
def try_few_shot_answer(section_text, question):
    prompt = f"""
You are a helpful chess tutor. Use the given section to answer the user's question.

You may infer the answer from the section even if it is not stated word-for-word.

{few_shot_examples}

Now here is the new question:

Section Text:
{section_text}

Question:
{question}

Answer in a simple sentence.
"""
    response = model.generate_content(prompt)
    return response.text.strip()

# 📄 Summarize section
def summarize_section(section_text, title):
    prompt = f"""You are a chess tutor summarizing this section for beginners.

Title: {title}

\"\"\"
{section_text[:4000]}
\"\"\"

Please summarize this section clearly.
"""
    response = model.generate_content(prompt)
    return response.text.strip()

# 🌐 Wikipedia fallback
def search_wikipedia(question):
    try:
        result = wikipedia.summary(question, sentences=2)
        return result
    except:
        return "❌ Wikipedia could not provide an answer."

# 🧪 Master function
def answer_question(section_title, question):
    display(Markdown(f"### ❓ Question: {question}"))

    section_text = sections.get(section_title)
    if not section_text:
        display(Markdown("❌ Section not found in document."))
        return

    # Step 1: Few-shot
    answer = try_few_shot_answer(section_text, question)
    if not answer or "not described" in answer.lower() or "not found" in answer.lower():
        display(Markdown("⚠️ Not clearly answered from section. Trying summary..."))

        # Step 2: Summarize and retry
        summary = summarize_section(section_text, section_title)
        answer = try_few_shot_answer(summary, question)

        if not answer or "not described" in answer.lower() or "not found" in answer.lower():
            display(Markdown("⚠️ Still unclear. Searching Wikipedia..."))

            # Step 3: Wikipedia fallback
            wiki_answer = search_wikipedia(question)
            display(Markdown(f"### 🌐 Wikipedia Answer:\n{wiki_answer}"))
            return

    # ✅ Final Answer
    display(Markdown(f"### ✅ Answer :\n{answer}"))


In [25]:
answer_question("Section 1. The Rules of the Game and a Few Checkmates", "What is check")


### ❓ Question: What is check

### ✅ Answer :
Check means an attack on the king.